In [ ]:
from urllib.parse import urlparse

import matplotlib.pyplot as plt
import xarray

import obstore
from zarr.storage import ObjectStore

from rio_tiler.io.xarray import XarrayReader

### MUR SST

In [ ]:
src_path = "https://mur-sst.s3.us-west-2.amazonaws.com/zarr-v1"

parsed = urlparse(src_path)
store = obstore.store.from_url(src_path, skip_signature=True)
zarr_store = ObjectStore(store=store, read_only=True)
ds = xarray.open_dataset(
    zarr_store,
    decode_times=True,
    decode_coords="all",
    consolidated=True,
    engine="zarr",
)

ds

In [ ]:
da = ds["analysed_sst"]
da

In [ ]:
da = ds["analysed_sst"]

# Make sure we have a valid CRS
crs = da.rio.crs or "epsg:4326"
da.rio.write_crs(crs, inplace=True)

# Select the first time stamp
da = da.isel(time=0)

with XarrayReader(da) as dst:
    print(dst.info())
    print(dst.minzoom, dst.maxzoom)

In [ ]:
with XarrayReader(da) as dst:
    img = dst.tile(31, 22, 6)

plt.imshow(img.data_as_image())

In [ ]:
with XarrayReader(da) as dst:
    img = dst.part(
        [-5, 45, 0.0, 49],
        max_size=1024,
    )

plt.imshow(img.data_as_image())

In [ ]:
geojson = {
    "type": "Feature",
    "properties": {},
    "geometry": {
        "coordinates": [
            [
                [-4.262517145703015, 49.58852771128011],
                [-6.121754821536115, 48.07366436953052],
                [-5.679508966604999, 47.26534294147169],
                [-2.890652452854056, 46.8101748147248],
                [-1.8527423498805433, 47.28371740880311],
                [-1.6541805026549241, 49.26565692911578],
                [-3.0440846882388257, 49.565118360747164],
                [-4.262517145703015, 49.58852771128011],
            ]
        ],
        "type": "Polygon",
    },
}

with XarrayReader(da) as dst:
    img = dst.feature(
        geojson,
        max_size=1024,
    )

plt.imshow(img.data_as_image())

### NetCDF 

In [ ]:
import fsspec

filesystem = fsspec.filesystem("https")


fp = filesystem.open(
    "https://dap.ceda.ac.uk/neodc/esacci/land_surface_temperature/data/MULTISENSOR_IRCDR/L3S/0.01/v2.00/monthly/2020/11/ESACCI-LST-L3S-LST-IRCDR_-0.01deg_1MONTHLY_DAY-20201101000000-fv2.00.nc"
)
ds = xarray.open_dataset(
    fp,
    engine="h5netcdf",
    decode_coords="all",
)
ds

In [ ]:
da = ds["lst"]

# # Make sure we have a valid CRS
crs = da.rio.crs or "epsg:4326"
da.rio.write_crs(crs, inplace=True)

# Select the first time stamp
# da = da.isel(time=0)
da

In [ ]:
with XarrayReader(da) as dst:
    print(dst.info())
    print(dst.minzoom, dst.maxzoom)

In [ ]:
with XarrayReader(da) as dst:
    img = dst.tile(31, 22, 6)

plt.imshow(img.data_as_image())